# Most of the code is from Keras Retinanet repository
https://github.com/fizyr/keras-retinanet/blob/master/examples/ResNet50RetinaNet.ipynb

# Make sure you have the proper environment from requirements.txt

In [0]:
!ls

sample_data


In [0]:
!git clone https://github.com/fizyr/keras-retinanet.git

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 5742, done.
remote: Total 5742 (delta 0), reused 0 (delta 0), pack-reused 5742
Receiving objects: 100% (5742/5742), 13.37 MiB | 4.88 MiB/s, done.
Resolving deltas: 100% (3855/3855), done.


In [0]:
%cd keras-retinanet/
!pip install . 

/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=169114 sha256=b1a2cfb70652c816da38676bd1afc6c368a90af3f4e732d26c93ae501367303d
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13346 sha256=4bfd7c7133faf0d492c554321b542203b299b603e13afa8834dbbf461ffc9e42
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet


In [0]:
!python setup.py build_ext --inplace

running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/keras_retinanet
creating build/temp.linux-x86_64-3.6/keras_retinanet/utils
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -c keras_retinanet/utils/compute_overlap.c -o build/temp.linux-x86_64-3.6/keras_retinanet/utils/compute_over

In [0]:
import os
import shutil
import zipfile
import urllib
import xml.etree.ElementTree as ET
import numpy as np
import csv
import pandas
from google.colab import drive
from google.colab import files

In [0]:
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

#### OPTION 1: DOWNLOAD INITIAL PRETRAINED MODEL FROM FIZYR ####
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)


('./snapshots/_pretrained_model.h5',
 <http.client.HTTPMessage at 0x7f8e77c39cc0>)

In [0]:
!ls snapshots/

_pretrained_model.h5


In [0]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%reload_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

# def get_session():
#     config = tf.compat.v1.ConfigProto()
#     config.gpu_options.allow_growth = True
#     return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
#keras.backend.tensorflow_backend.set_session(tf.compat.v1.keras.backend.get_session())

Using TensorFlow backend.


In [0]:
print(cv2.__version__)

4.1.2


In [0]:
model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])
print(model_path)

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')


snapshots/_pretrained_model.h5
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
      

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls

build		 images		  README.md	    setup.py
CONTRIBUTORS.md  keras_retinanet  requirements.txt  snapshots
examples	 LICENSE	  setup.cfg	    tests


In [0]:
! ls /content/gdrive/'My Drive'/'Colab Notebooks'/EdgeAIContest3
!unlink EdgeAIContest3
!ln -s /content/gdrive/'My Drive'/'Colab Notebooks'/EdgeAIContest3 ./

data  LICENSE  model  notebook	README.md  requirements.txt  src
unlink: cannot unlink 'EdgeAIContest3': No such file or directory


In [0]:
!ls

build		 examples	  LICENSE	    setup.cfg  tests
CONTRIBUTORS.md  images		  README.md	    setup.py
EdgeAIContest3	 keras_retinanet  requirements.txt  snapshots


In [0]:
!pip install -r EdgeAIContest3/requirements.txt

     |████████████████████████████████| 92kB 4.2MB/s 
     |████████████████████████████████| 122kB 18.5MB/s 
     |████████████████████████████████| 788kB 10.9MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 112kB 36.1MB/s 
     |████████████████████████████████| 7.8MB 28.1MB/s 
     |████████████████████████████████| 9.7MB 40.2MB/s 
     |████████████████████████████████| 28.2MB 97kB/s 
     |████████████████████████████████| 2.1MB 35.2MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 358kB 38.2MB/s 
     |████████████████████████████████| 1.3MB 29.1MB/s 
     |████████████████████████████████| 921kB 43.4MB/s 
     |████████████████████████████████| 512kB 44.3MB/s 
     |████████████████████████████████| 276kB 32.9MB/s 
     |████████████████████████████████| 501kB 44.7MB/s 
     |████████████████████████████████| 133kB 46.8MB/s 
  Created wheel for prometheus-client: filename=prometh

In [0]:
!ls -lha EdgeAIContest3/data/train_00.json
!ls -lha EdgeAIContest3/data/train_00.mp4

-rw------- 1 root root 428K Mar 26 14:16 EdgeAIContest3/data/train_00.json
-rw------- 1 root root 88M May 19 12:18 EdgeAIContest3/data/train_00.mp4


In [0]:
import json
current_file = "train_01"
with open("EdgeAIContest3/data/train_annotations/{}.json".format(current_file)) as f:
    train_json = json.load(f)

In [0]:
cap = cv2.VideoCapture("EdgeAIContest3/data/train_videos/{}.mp4".format(current_file))
w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("EdgeAIContest3/data/train_videos/{}_output.mp4".format(current_file), fourcc, 15.0, (int(w), int(h)))

# Check if camera opened successfully
if (cap.isOpened()== False):
  print("Error opening video stream or file")

In [0]:
# use this variable if you want to process first n frames and uncomment the frame_count condition
frame_count=0

# Read until video is completed
while (cap.isOpened()):
    # if (frame_count == 10):
    #     break
    # Capture frame-by-frame
    ret, image = cap.read()
    if ret == True:
        draw = image.copy()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
         # preprocess image for network
        image = preprocess_image(image)
        image, scale = resize_image(image)

        # process image
        start = time.time()
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
        print("processing time: ", time.time() - start)

        # correct for image scale
        boxes /= scale

        # visualize detections
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            # scores are sorted so we can break
            ## Just selecting label=0 (Pedestrian for now)
            if score < 0.5 or not (label==0):
                break

            color = label_color(label)

            b = box.astype(int)
            draw_box(draw, b, color=color)

            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)
        
        # visualize ground truth
        if 'Pedestrian' in train_json['sequence'][frame_count].keys():
            pedestrians_list = train_json['sequence'][frame_count]['Pedestrian']
        else:
            pedestrians_list=[]
        for box in pedestrians_list:
            b = box['box2d']
            b = list(map(int, b))
            draw_box(draw, b, color=(255, 0, 0))
            draw_caption(draw, b, "P")
        
        frame_count = frame_count + 1
        # write the resulting frame
        out.write(draw)
 
    # Break the loop
    else:
        break

processing time:  0.22180628776550293
processing time:  0.15398311614990234
processing time:  0.15438365936279297
processing time:  0.1564502716064453
processing time:  0.1554431915283203
processing time:  0.1571204662322998
processing time:  0.15438151359558105
processing time:  0.15029621124267578
processing time:  0.1636204719543457
processing time:  0.15557479858398438
processing time:  0.15861034393310547
processing time:  0.15846776962280273
processing time:  0.15577483177185059
processing time:  0.15679931640625
processing time:  0.154799222946167
processing time:  0.15984082221984863
processing time:  0.15600800514221191
processing time:  0.1553952693939209
processing time:  0.15736055374145508
processing time:  0.15461182594299316
processing time:  0.15482044219970703
processing time:  0.15546631813049316
processing time:  0.15642189979553223
processing time:  0.15311312675476074
processing time:  0.15711116790771484
processing time:  0.15484857559204102
processing time:  0.15

In [0]:
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

# write the file
out.release()

In [0]:
def process_video(video_id):
  with open("EdgeAIContest3/data/train_annotations/{}.json".format(video_id)) as f:
    train_json = json.load(f)
  cap = cv2.VideoCapture("EdgeAIContest3/data/train_videos/{}.mp4".format(video_id))
  w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
  h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
  

  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter("EdgeAIContest3/data/train_videos/{}_output.mp4".format(video_id), fourcc, 15.0, (int(w), int(h)))

  # Check if camera opened successfully
  if (cap.isOpened()== False):
    print("Error opening video stream or file")
  # use this variable if you want to process first n frames and uncomment the frame_count condition
  frame_count=0

  # Read until video is completed
  while (cap.isOpened()):
      # if (frame_count == 10):
      #     break
      # Capture frame-by-frame
      ret, image = cap.read()
      if ret == True:
          draw = image.copy()
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          # preprocess image for network
          image = preprocess_image(image)
          image, scale = resize_image(image)

          # process image
          start = time.time()
          boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
          print("processing time: ", time.time() - start)

          # correct for image scale
          boxes /= scale

          # visualize detections
          for box, score, label in zip(boxes[0], scores[0], labels[0]):
              # scores are sorted so we can break
              ## Just selecting label=0 (Pedestrian for now)
              if score < 0.5 or not (label==0):
                  break

              color = label_color(label)

              b = box.astype(int)
              draw_box(draw, b, color=color)

              caption = "{} {:.3f}".format(labels_to_names[label], score)
              draw_caption(draw, b, caption)
          
          # visualize ground truth
          if 'Pedestrian' in train_json['sequence'][frame_count].keys():
              pedestrians_list = train_json['sequence'][frame_count]['Pedestrian']
          else:
              pedestrians_list=[]
          for box in pedestrians_list:
              b = box['box2d']
              b = list(map(int, b))
              draw_box(draw, b, color=(255, 0, 0))
              draw_caption(draw, b, "P")
          
          frame_count = frame_count + 1
          # write the resulting frame
          out.write(draw)
  
      # Break the loop
      else:
          break
  # When everything done, release the video capture object
  cap.release()
  
  # Closes all the frames
  cv2.destroyAllWindows()

  # write the file
  out.release()

In [0]:
process_video("train_02")

processing time:  0.2149806022644043
processing time:  0.16411042213439941
processing time:  0.15933442115783691
processing time:  0.15356659889221191
processing time:  0.15807366371154785
processing time:  0.1536426544189453
processing time:  0.15247535705566406
processing time:  0.1515507698059082
processing time:  0.15422725677490234
processing time:  0.15411710739135742
processing time:  0.15418076515197754
processing time:  0.15700674057006836
processing time:  0.15546655654907227
processing time:  0.15761756896972656
processing time:  0.15587711334228516
processing time:  0.15674829483032227
processing time:  0.1570143699645996
processing time:  0.15877127647399902
processing time:  0.16711640357971191
processing time:  0.15503406524658203
processing time:  0.15490508079528809
processing time:  0.15456652641296387
processing time:  0.15449237823486328
processing time:  0.155562162399292
processing time:  0.15417027473449707
processing time:  0.1512305736541748
processing time:  0